## Step 1. 加载模型与Tokenizer

In [30]:
data = {
  "client0_attacker_14_normal_step": 0.5175706596299099,
  "client0_attacker_19_normal_step": 0.5429864203393666,
  "client0_attacker_32_normal_step": 0.3857142807290817,
  "_step": 4249,
  "client0_loss": 2.031713839054108,
  "client0_attacker_29_tr2t_avg": 0.5275647235774503,
  "client0_attacker_8_normal_avg": 0.6366766778182428,
  "client0_attacker_16_normal_step": 0.5285769033264401,
  "client0_attacker_24_normal_step": 0.544871789879511,
  "client0_attacker_28_normal_step": 0.4467532417557092,
  "client0_test-ppl": 14.366519927978516,
  "client0_attacker_0_normal_avg": 0.5362440276624566,
  "client0_attacker_32_normal_avg": 0.4366247128481753,
  "client0_attacker_9_normal_step": 0.5223380441441088,
  "client0_attacker_34_normal_avg": 0.31704178424232093,
  "client0_attacker_35_normal_avg": 0.27924916250140014,
  "client0_attacker_30_normal_step": 0.4590469049036024,
  "global_step": 4249,
  "client0_global_round": 0,
  "client0_attacker_5_b2tr_avg": 0.6304907277186129,
  "client0_attacker_3_normal_avg": 0.6183885509769569,
  "client0_attacker_23_normal_step": 0.5818181768206444,
  "client0_attacker_31_normal_step": 0.41103848447329144,
  "client0_self_pt": 0,
  "client0_attacker_11_normal_avg": 0.6377722236693276,
  "client0_attacker_33_normal_avg": 0.39310278683275807,
  "client0_attacker_6_normal_step": 0.5439360879591302,
  "client0_attacker_2_normal_avg": 0.6089481650353057,
  "client0_attacker_4_normal_avg": 0.6238578184133798,
  "client0_attacker_22_normal_step": 0.5893812020286877,
  "client0_attacker_25_normal_step": 0.5504761854822313,
  "client0_attacker_13_normal_step": 0.5263951684539971,
  "client0_attacker_9_normal_avg": 0.6384095213150243,
  "client0_attacker_16_normal_avg": 0.6330719245495596,
  "client0_attacker_24_normal_avg": 0.5943958147410345,
  "client0_attacker_26_normal_avg": 0.5723204183075637,
  "client0_attacker_15_normal_step": 0.5175706596299099,
  "client0_attacker_5_b2tr_step": 0.5145243232532479,
  "client0_attacker_7_normal_avg": 0.6351210251036679,
  "client0_attacker_12_normal_avg": 0.6366890118070682,
  "client0_attacker_13_normal_avg": 0.6347554874791649,
  "client0_attacker_21_normal_avg": 0.620009580633759,
  "client0_attacker_25_normal_avg": 0.5867858649221694,
  "client0_attacker_27_normal_avg": 0.5616850112415854,
  "client0_attacker_3_normal_step": 0.5351071642617553,
  "_wandb.runtime": 1649,
  "client0_local_epoch": 0,
  "client0_attacker_10_normal_avg": 0.6380667878549796,
  "client0_attacker_20_normal_avg": 0.624557655192967,
  "client0_attacker_20_normal_step": 0.5729323258308399,
  "client0_attacker_21_normal_step": 0.5688311638336314,
  "client0_attacker_33_normal_step": 0.33369407870222145,
  "client0_attacker_34_normal_step": 0.30359476626650117,
  "client0_attacker_31_normal_avg": 0.4747830768028701,
  "client0_attacker_10_normal_step": 0.5096798162960074,
  "client0_attacker_12_normal_step": 0.5175706596299099,
  "client0_attacker_26_normal_step": 0.5203007468834715,
  "_runtime": 1651.2562370300293,
  "_timestamp": 1707696679.616939,
  "client0_attacker_1_normal_step": 0.5177280500885659,
  "client0_attacker_28_normal_avg": 0.5459204030441271,
  "client0_attacker_6_normal_avg": 0.6324742085562801,
  "client0_attacker_17_normal_avg": 0.6309008512450668,
  "client0_attacker_18_normal_avg": 0.6301045366130664,
  "client0_attacker_4_normal_step": 0.5473022862129748,
  "client0_attacker_17_normal_step": 0.5469824243357921,
  "client0_attacker_15_normal_avg": 0.6324732529184328,
  "client0_attacker_22_normal_avg": 0.611303527540108,
  "client0_attacker_23_normal_avg": 0.6043506538926331,
  "client0_attacker_7_normal_step": 0.5562817669714759,
  "client0_self": 0.0005098648707545071,
  "client0_local_step": 4249,
  "client0_attacker_8_normal_step": 0.5549242374281007,
  "client0_attacker_29_tr2t_step": 0.454184699192842,
  "client0_attacker_2_normal_step": 0.5012778336386043,
  "client0_attacker_11_normal_step": 0.5135746556334841,
  "client0_attacker_0_normal_step": 0.5027100221139359,
  "client0_attacker_14_normal_avg": 0.6337987627417009,
  "client0_attacker_35_normal_step": 0.2107142807178891,
  "client0_attacker_27_normal_step": 0.5097402547483976,
  "client0_attacker_1_normal_avg": 0.5966522677713505,
  "client0_attacker_19_normal_avg": 0.627233252992626,
  "client0_attacker_30_normal_avg": 0.5104403879249436,
  "client0_attacker_18_normal_step": 0.5510835863330007
}

In [1]:

import os
import sys

sys.path.append(os.path.abspath('../..'))
from sfl import config
from transformers import AutoTokenizer
from sfl.simulator.dataset import WikiTextFedDataset

tokenizer = AutoTokenizer.from_pretrained(os.path.join(config.model_download_dir, "gpt2/"))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 50256
ds = WikiTextFedDataset(tokenizer, ['0', '1'])
dl = ds.get_dataloader('0',batch_size=2)
for x in dl:
    print(x)
    break

tens!
tens!
tens!
ntens!
tens!
{'input_ids': tensor([[  257,  2196,  1262,   281,   477,  2488,    12,    31,  2042,  3350,
           351,  1279,  2954,    29, 14926,  1279,  2954,    29,   355,  3619,
           837,   370,   918,   309,    13,  4373,   355,   978,    70,  1142,
           261,   837,  5506,   775, 25900,   355, 11182,  1709,   441, 10076,
           837,  1279,  2954,    29, 29045,   355, 23367,   813,   837,  5180,
          4889, 41788,   355, 39661,   437,  8622,   837,  1279,  2954,    29,
          1279,  2954,    29,   355,  4544, 35417,   837,   290,  1279,  2954,
            29, 15722,   355,  9356,   609,   292, 26664,   837,   900,   287,
           262,  1578,  1829,   764, 15416, 13612,   837,   281,  3594,  3437,
           508,   550,  4271, 16573,  1052, 41765, 13895,  3903,   416, 45622,
           837,   925,   262,  6244,  2646,  2162,   340,  5788, 18373,   376,
          3333,   357,  3619,  1267,   837, 34315, 36815,   357,   978,  1360,
       

In [14]:

import os
import sys


sys.path.append(os.path.abspath('../..'))
from sfl.model.bert.bert_wrapper import BertForSequenceClassificationSplitModel
from sfl import config

model = BertForSequenceClassificationSplitModel.from_pretrained(os.path.join(config.model_download_dir, "google-bert/bert-base-uncased/"))

Some weights of BertForSequenceClassificationSplitModel were not initialized from the model checkpoint at /root/autodl-tmp/sfl/models/google-bert/bert-base-uncased/ and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from sfl.config import FLConfig
config = FLConfig(global_round=50,
                  client_epoch=2,  # 每轮联邦每个Client训2轮
                  split_point_1=3,
                  split_point_2=10,  # [0,1 | 2,3,.... 29| 30, 31]
                  use_lora_at_trunk=True,  # 在trunk部分使用LoRA
                  top_and_bottom_from_scratch='False',  # top和bottom都不采用预训练参数.
                  noise_mode="dxp",
                  noise_scale=5.0,  # 噪声大小
                  use_lora_at_top=False,
                  use_lora_at_bottom=False
                  )

model.config_sfl(config, None)

In [16]:
model = model.convert_to_lora_model()

In [22]:
model.config

BertConfig {
  "_name_or_path": "/root/autodl-tmp/sfl/models/google-bert/bert-base-uncased/",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [17]:

import os
import sys

sys.path.append(os.path.abspath('../..'))
from sfl.utils.training import get_best_gpu
from sfl.model.gpt2.gpt2_split import GPT2SplitLMHeadModel
from transformers import AutoTokenizer
from sfl import config

device = get_best_gpu()
tokenizer = AutoTokenizer.from_pretrained(os.path.join(config.model_download_dir, "gpt2/"))
model = GPT2SplitLMHeadModel.from_pretrained(os.path.join(config.model_download_dir, "gpt2-large/"))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = 50256
model.to(device)

KeyboardInterrupt: 

In [8]:
from sfl.utils.model import generate


# 测试模型输出
def get_output(text, md=model):
    t = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    res = model(t['input_ids'].to(md.device), attention_mask=t['attention_mask'].to(md.device))
    r = tokenizer.decode(res.logits.argmax(dim=-1)[-1], skip_special_tokens=True)
    return r


print(generate("如何自杀", tokenizer, model))

如何自杀的话,但那么就没有可能会让他们绝对于这种情况下来说不定,而且比较好了吧。 "I see, so that's how it is. Then, I'll leave it to you, Your Majesty." 「当然,非常感谢。首先抱歉,魔导王也不会选择」 "Understood!" 「遵命!」 Remedios bowed deeply. 蕾梅蒂欧丝发出现在圣王国的视线贝德的提问题,只要接着古斯塔博打算的反应。 After that, the two of them left the room. 安兹确实际上前进行,涅娅将亚乌菈以及被深处交消失的笑容。 As they left, Ainz could not help but feel a sense of déjà vu, as though he had seen the same scene before. 为此时候是知道一点的想法,虽然恐怕恢复的意思。 He had not seen it before, and he did not want to see it again. However, that was not the point. The point was that this was the first time he was seeing this scene in his entire life. 然后,还是更多少并不清楚。不过,从脑海和哪里的人类等待。若是身体的心中,完全不�


## Step 2. 加载攻击模型

In [ ]:

from sfl.config import attacker_path
from sfl.utils.training import get_attacker_class, extract_attacker_path

attack_model = 'gru'
# 加载攻击模型
attacker_cls = get_attacker_class(attack_model)
attacker_path_1, attacker_path_2 = extract_attacker_path(
    {'split_point_1': 2, 'split_point_2': 10, 'attacker_path': attacker_path})
attacker = attacker_cls.from_pretrained(attacker_path_1)
attacker2 = attacker_cls.from_pretrained(attacker_path_2)
#
# attacker = GRUAttackModel.from_pretrained(
#     '/root/autodl-tmp/sfl/models/attacker/gpt2/piqa/train*1.000-test*1.000/gru/b2tr-2/epoch_19_rouge_0.9849')
# attacker2 = GRUAttackModel.from_pretrained(
#     '/root/autodl-tmp/sfl/models/attacker/gpt2/piqa/train*1.000-test*1.000/gru/tr2t-10/epoch_19_rouge_0.9261')

## Step 3. 设置联邦训练流程

In [6]:
from sfl.config import FLConfig
config = FLConfig(global_round=50,
                  client_epoch=2,  # 每轮联邦每个Client训2轮
                  split_point_1=3,
                  split_point_2=32,  # [0,1 | 2,3,.... 29| 30, 31]
                  use_lora_at_trunk=True,  # 在trunk部分使用LoRA
                  top_and_bottom_from_scratch='False',  # top和bottom都不采用预训练参数.
                  noise_mode="dxp",
                  noise_scale=5.0,  # 噪声大小
                  )

model.config_sfl(config, None)
model.print_split_model()

=================Split-/root/autodl-tmp/sfl/models/gpt2-large/=================
==================Top Layers==================

transformer.h.32:[ln_1.weight: (1280,), ln_1.bias: (1280,), attn.c_attn.weight: (1280, 3840), attn.c_attn.bias: (3840,), attn.c_proj.weight: (1280, 1280), attn.c_proj.bias: (1280,), ln_2.weight: (1280,), ln_2.bias: (1280,), mlp.c_fc.weight: (1280, 5120), mlp.c_fc.bias: (5120,), mlp.c_proj.weight: (5120, 1280), mlp.c_proj.bias: (1280,)]

transformer.h.33:[ln_1.weight: (1280,), ln_1.bias: (1280,), attn.c_attn.weight: (1280, 3840), attn.c_attn.bias: (3840,), attn.c_proj.weight: (1280, 1280), attn.c_proj.bias: (1280,), ln_2.weight: (1280,), ln_2.bias: (1280,), mlp.c_fc.weight: (1280, 5120), mlp.c_fc.bias: (5120,), mlp.c_proj.weight: (5120, 1280), mlp.c_proj.bias: (1280,)]

transformer.h.34:[ln_1.weight: (1280,), ln_1.bias: (1280,), attn.c_attn.weight: (1280, 3840), attn.c_attn.bias: (3840,), attn.c_proj.weight: (1280, 1280), attn.c_proj.bias: (1280,), ln_2.weight:

In [2]:
import torch
from sfl.utils.model import calculate_rouge
from sfl.config import FLConfig
from sfl.simulator.simulator import SFLSimulator
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm_notebook
from sfl.model.split_model import SplitModel
from sfl.simulator.strategy import FLStrategy
from sfl.simulator.dataset import PIQAFedDataset
from torch.optim import AdamW


# 定义Client本地学习策略
class QAFLStrategy(FLStrategy):

    def __init__(self):
        super().__init__()
        self.attacker_rouge_b2tr = []
        self.attacker_rouge_tr2t = []
        self.client_logs = {}

    def client_step(self, global_round, client_id: str, llm: SplitModel, dataloader: DataLoader, cfg: FLConfig):
        optimizer = AdamW(llm.parameters(), lr=1e-5)
        with tqdm_notebook(total=cfg.client_epoch * len(dataloader)) as pbar:
            for epoch in range(cfg.client_epoch):
                for step, batch in enumerate(dataloader):
                    optimizer.zero_grad()
                    input_ids = batch['input_ids'].to(llm.device)
                    attention_mask = batch['input_att_mask'].to(llm.device)
                    outputs = llm(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)
                    self.fp_done(client_id, epoch, step, batch)  # Collect intermediate results
                    loss = outputs.loss
                    pbar.set_description(f'Client {client_id} Epoch {epoch} Loss {loss.item():.3f}')
                    loss.backward()
                    self.bp_done(client_id, epoch, step, batch)  # Collect gradients
                    optimizer.step()
                    pbar.update(1)
                avg_rouge1 = sum([r["rouge-l"]["f"] for r in self.attacker_rouge_b2tr]) / len(self.attacker_rouge_b2tr)
                print(f'ATTACK! Bottom-trunk, Client {client_id} Epoch {epoch} RougeL {avg_rouge1:.3f}')
                avg_rouge2 = sum([r['rouge-l']['f'] for r in self.attacker_rouge_tr2t]) / len(self.attacker_rouge_tr2t)
                print(f'ATTACK! Trunk-Top, Client {client_id} Epoch {epoch} RougeL {avg_rouge2:.3f}')
                self.client_logs.setdefault(client_id, {})
                self.client_logs[client_id][epoch] = {"bottom-trunk": avg_rouge1, "trunk-top": avg_rouge2}
                self.attacker_rouge_b2tr.clear()
                self.attacker_rouge_tr2t.clear()

    def aggregation_step(self, global_round, params):
        report = {}
        report['global_round'] = global_round
        for cid, epochs in self.client_logs.items():
            for epc, rep in epochs.items():
                for k, v in rep.items():
                    report[f'client{cid}-epoch{epc}-{k}'] = v
        wandb.log(report)
        print(report)
        self.client_logs = {}
        return super(QAFLStrategy, self).aggregation_step(global_round, params)

    def callback_fp_param(self, client_id, local_epoch, local_step, b2tr_params, tr2t_params, batch):
        #  这里获取某epoch、step中，前传过程的两次传输参数，b2tr(bottom-trunk), tr2t(trunk-top)
        with torch.no_grad():
            rouge_res_b2tr = calculate_rouge(tokenizer, attacker.search(b2tr_params, model), batch['input_text'],
                                             is_tokens=True)
            rouge_res_tr2t = calculate_rouge(tokenizer, attacker2.search(tr2t_params, model), batch['input_text'],
                                             is_tokens=True)
            self.attacker_rouge_b2tr.append(rouge_res_b2tr)
            self.attacker_rouge_tr2t.append(rouge_res_tr2t)
            print(
                f'ATTACK! Bottom-trunk, Client {client_id} Epoch {local_epoch} Step {local_step} RougeL {rouge_res_b2tr["rouge-l"]["f"]:.3f}')

    def callback_bp_param(self, client_id, local_epoch, local_step, t2tr_params, tr2b_params, batch):
        #  这里获取某epoch、step中，反传过程的两次传输参数
        pass


client_ids = [str(i) for i in range(3)]
config = FLConfig(global_round=50,
                  client_epoch=2,  # 每轮联邦每个Client训2轮
                  split_point_1=2,
                  split_point_2=10,  # [0,1 | 2,3,.... 29| 30, 31]
                  use_lora_at_trunk=True,  # 在trunk部分使用LoRA
                  top_and_bottom_from_scratch='False',  # top和bottom都不采用预训练参数.
                  noise_mode="dxp",
                  noise_scale=5.0,  # 噪声大小
                  )
fed_dataset = PIQAFedDataset(tokenizer=tokenizer, client_ids=client_ids, shrink_frac=0.15)
simulator = SFLSimulator(client_ids=client_ids, strategy=QAFLStrategy(), llm=model, tokenizer=tokenizer,
                         dataset=fed_dataset, config=config)
model.print_split_model()
attacker.to(model.device)
attacker2.to(model.device)

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/piqa/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011 (last modified on Wed Jan 10 12:49:01 2024) since it couldn't be found locally at piqa., or remotely on the Hugging Face Hub.


TypeError: Can't instantiate abstract class QAFLStrategy with abstract methods callback_intermediate_result, client_evaluate

## Step 3. 开始联邦模拟

In [5]:
import wandb

wandb.init(
    project="sfl-with-attacker",
    # track hyperparameters and run metadata
    config={
        "dataset": 'piqa',
        "attacker": "piqa-validation",
        "noise": "2.0"
    }
)

simulator.simulate()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: stupidtree. Use `wandb login --relogin` to force relogin


==================================Global Round 0=================================


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/94 [00:00<?, ?it/s]

ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 0 RougeL 0.955
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 1 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 2 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 3 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 4 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 5 RougeL 0.967
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 6 RougeL 0.986
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 7 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 8 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 9 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 10 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 11 RougeL 1.000
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 12 RougeL 0.974
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 13 RougeL 0.942
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 14 RougeL 0.939
ATTACK! Bottom-trunk, Client 2 Epoch 0 Step 15 RougeL 1.000



KeyboardInterrupt



In [ ]:
from sfl.utils.model import sentence_score

sentence_score("I'm fine, thank you!", model, tokenizer)